In [ ]:
import music21 as mu
import pandas as pd
import numpy as np
from data_helper import *

def flength(time):
    """
    打鍵時間をquarterLengthに変換
    """
    threasholds = [0.099, 0.18, 0.30, 0.47, 0.60, 0.81, 1.00, 1.15, 1.30, 1.60, 1.90]
    lengths     = [0.125, 0.25, 0.50, 0.75, 1.00, 1.25, 1.50, 1.75, 2.00, 2.25, 2.50]
    for i in range(len(threasholds)):
        if time < threasholds[i]:
            return lengths[i]
    return 4 # whole

def fnote(name, t):
    """
    音名と打鍵時間からNoteを生成
    """
    return mu.note.Note(name=name, quarterLength=flength(t))

def count_match(notes, fingers):
    """
    PIGデータと運指配列との一致数を返す
    """
    match = 0
    for i in range(len(notes)):
        if notes.iloc[i].finger == fingers.iat[i]:
            match += 1
    return match

def appendPIGNotes(stream, pig_notes, fingers):
    time = 0.0
    for i in range(len(pig_notes)):
        c = pig_notes.iloc[i]
        if(time < c.t0 and c.t0 - time > 1/32):
            r = mu.note.Rest(quarterLength=flength(c.t0 - time))
            stream.append(r)
        n = fnote(c.note, c.t1 - c.t0)
        n.articulations.append(mu.articulations.Fingering(fingers.iat[i]))
        if c.finger != fingers.iat[i]:
            n.style.color = "red"
            n.addLyric(c._finger)
        stream.append(n)
        time = c.t1

def drawMatches(stream, notes, fingerZipped):
    time = 0.0
    for i, fingers in enumerate(fingerZipped):
        c = notes.iloc[i]
        if(time < c.t0 and c.t0 - time > 1/32):
            r = mu.note.Rest(quarterLength=flength(c.t0 - time))
            stream.append(r)
        n = fnote(c.note, c.t1 - c.t0)
        differ = (len(set(fingers)) - 1) / (len(fingers) - 1)
        color = int((1 - pow(1 - differ, 2)) * 255)
        red = "{:02x}".format(color)
        n.style.color = f"#{red}0000"
        stream.append(n)
        time = c.t1

In [ ]:
convert_fingering_to_PIG(
    './data/001-my_fingering.csv',
    './data/001-my_fingering_R.txt', 
    './data/001-my_fingering_L.txt', 
    "./tmp/PianoFingeringDataset/FingeringFiles/001-1_fingering.txt")


In [ ]:
table = read_PIG("./tmp/PianoFingeringDataset/FingeringFiles/001-1_fingering.txt")
table2 = read_PIG("./tmp/PianoFingeringDataset/FingeringFiles/001-2_fingering.txt")
table5 = read_PIG("./tmp/PianoFingeringDataset/FingeringFiles/001-5_fingering.txt")
table8 = read_PIG("./tmp/PianoFingeringDataset/FingeringFiles/001-8_fingering.txt")

czernyR = read_fingering('./data/001-Czerny_fingering_R.txt')
czernyL = read_fingering('./data/001-Czerny_fingering_L.txt')

In [ ]:
rights = table[table.ch == 0]
pr = mu.stream.Part()
appendPIGNotes(pr, rights, czernyR)

lefts = table[table.ch == 1]
pl = mu.stream.Part()
appendPIGNotes(pl, lefts, czernyL)

score = mu.stream.Score()
score.insert(0, pr)
score.insert(0, pl)

score.write('musicxml', './mxl/bach_invention1.xml')
#m = count_match(rights, czernyR) + count_match(lefts, czernyL)
#print(f"{m} / {len(rights) + len(lefts)}")

In [ ]:
#fingersr = zip(table[table.ch == 0].finger, table2[table.ch == 0].finger, table5[table.ch == 0].finger, table8[table.ch == 0].finger)
#fingersl = zip(table[table.ch == 1].finger, table2[table.ch == 1].finger, table5[table.ch == 1].finger, table8[table.ch == 1].finger)

rights = table[table.ch == 0]
pr = mu.stream.Part()
drawMatches(pr, rights, zip(table[table.ch == 0].finger, czernyR))

score = mu.stream.Score()
score.insert(0, pr)
# score.write('musicxml', './mxl/PIG_differs2.xml')

#m = match(rights, czernyR) + match(lefts, czernyL)
#print(f"{m} / {len(rights) + len(lefts)}")

In [ ]:
def make_match_matrix(pig_array):
    df = pd.DataFrame(columns = range(len(pig_array)))
    pig_fingers = [table.finger for table in pig_array]
    data_length = len(pig_fingers[0])
    for i in range(len(pig_fingers)):
        row = {}
        for j in range(len(pig_fingers)):
            fingersr = zip(pig_fingers[i], pig_fingers[j]) # make pair
            cnt = 0
            for _, fs in enumerate(fingersr):
                if len(set(fs)) < len(fs):
                    cnt += 1
            row[j] = cnt / data_length
            #rate = "{:.3f}".format(cnt / data_length)
        df = df.append(row, ignore_index = True)
    print(df)

pig = [table, table2, table5, table8]
make_match_matrix(pig)